## Transmitter

In [1]:
from optical_flax.generate_data import Tx_data, channel, Rx_data
import jax
import jax.random as rd

In [3]:
key = rd.PRNGKey(123)
batch = 10
sigWDM, symbWDM, param = Tx_data(key, batch, Nch=11, Power=0, SpS=16)

Transmitter is working..
Sample rate fa: 5.76e+11, Cut off frequency fc: 2.75e+11, fa > 2fc: True
signal shape: (10, 1600000, 2), symb shape: (10, 100000, 11, 2)


## Channel

In [4]:
Fs = param.Rs*param.SpS  # sample rates
sigWDM_rx, paramCh = channel(sigWDM, Fs)

data transmition...
channel transmission done!
Signal shape (10, 1600000, 2)


## Receiver

In [6]:
key = rd.PRNGKey(234)
rx_sps = 8
data_sml = Rx_data(key, sigWDM_rx, symbWDM, rx_sps, param=param, paramCh=paramCh)

In [13]:
from collections import namedtuple
import pickle

from optical_flax.utils import auto_rho
DataInput = namedtuple('DataInput', ['y', 'x', 'w0', 'a'])